<a href="https://colab.research.google.com/github/Tran-Thu-Le/SFW/blob/main/colabsetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to SFW Project

SFW stands for Sliding Frank_Wolfe algorithm for solving Blasso problem.

Blasso problem is the following 

\begin{align}
\min_{\mu} \frac{1}{2}||y-\mathscr A \mu||^2+\lambda ||\mu||_{TV}
\end{align}

This notebook should be run on Google Colab by clicking [here](https://colab.research.google.com/github/Tran-Thu-Le/SFW/blob/main/intro.ipynb).
This notebook is used to create Python scripts when run on Colab.

## Clone the repo

For first time run, please remove `#` at the second command and then put it back.

In [1]:
!git clone https://github.com/Tran-Thu-Le/SFW
#%cd SFW

Cloning into 'SFW'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.
/content/SFW


# Objects

Run the code cells in this section to setup objects for SFW.

### kernels

#### gauss

In [6]:
import numpy as np
from scipy.integrate import quad

class gauss1d(object):
  def __init__(self, domain_meas, domain_l2f):
    self.domain_meas=domain_meas
    self.domain_l2f=domain_l2f
    self.w=0.2

  def param(self, arg):
    return lambda x: 2**(-(x-arg)**2/self.w**2)

  def norm(self, arg):
    return quad(self.param(arg), self.domain_l2f[0], self.domain_l2f[1])[0]



if __name__=="__main__":
  domain_l2f=np.array([0., 1.])
  domain_meas=np.array([0., 1.])
  g=gauss1d(domain_meas, domain_l2f)
  print(g.norm(0.5))




0.4244051399932198


### l2function

In [4]:
%%writefile objects/l2function.py
import numpy as np

class l2function(object):
  def __init__(self, vector, domain_l2f):
    self.vector=vector
    self.domain=domain

    self.dim_domain=1
    self.grid_size=len(self.vector)
    self.grid=np.linspace(self.domain[0], self.domain[-1], self.grid_size) 
    self.function=lambda t: np.interp(t, self.grid, self.vector)

if __name__=="__main__":
  domain=[0., 1.]
  grid=np.linspace(domain[0], domain[1], 10) 
  vector=grid**2
  f=l2function(vector, domain)
  print("attribute", f.grid)
  print("function", f.function(0.5))


Writing objects/l2function.py


### measure

In [3]:
#%%writefile objects/measure.py
import numpy as np

class measure(object):
  def __init__(self, coeffs, args, domain_meas):
    assert(isinstance(coeffs, np.ndarray))
    assert(isinstance(args, np.ndarray))
    assert(isinstance(domain_meas, np.ndarray))
    assert(len(coeffs) == args.shape[1] )
    assert(args.shape[0] == domain_meas.shape[0])
    assert(domain_meas.shape[1]==2 )

    self.coeffs=coeffs
    self.args=args
    self.domain_meas=domain_meas

    self.dim_arg=self.args.shape[0]
    self.len=len(self.coeffs)
    self.item=lambda i: (self.coeffs[i], self.args[:, i].reshape(-1,1))
    self.norm=sum(np.abs(self.coeffs))

if __name__=="__main__":
  domain_meas=np.array([[0., 0.1], [1., 0.5]])
  args=np.array([[0., 0.3, 0.6], [0.2, 0.2, 0.2]]) 
  coeffs=np.array([1., 1., 1.])
  print(args)
  mu=measure(coeffs, args, domain_meas)
  print("attribute1", mu.item(-1))
  print("attribute2", mu.norm)

[[0.  0.3 0.6]
 [0.2 0.2 0.2]]
attribute1 (1.0, array([[0.6],
       [0.2]]))
attribite2 3.0


### contfunction

### atom